Ce notebook illustre comment :
1. Charger et préparer les données (articles + interactions).
2. Implémenter deux systèmes de recommandation : 
     - Collaborative Filtering (user-based kNN) 
     - Content-Based Filtering (similarité embeddings).
3. Comparer sommairement les performances (hit rate).
4. Sauvegarder les modèles et données transformées au format pickle.
5. Fournir des fonctions réutilisables pour la prédiction 
   dans un environnement serverless (Azure Functions) ou Streamlit.

In [1]:
import glob
import pickle
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# scipy.sparse pour la matrice sparse
from scipy.sparse import coo_matrix, csr_matrix

# Pour désactiver certains warnings éventuels
import warnings
warnings.filterwarnings('ignore')

# Configuration

In [2]:
# Paths vers les données
PATH_ARTICLES_EMBEDDINGS = "../data/articles_embeddings.pickle"
PATH_ARTICLES_METADATA   = "../data/articles_metadata.csv"
PATH_CLICKS_PATTERN      = "../data/clicks/clicks_hour_*.csv"  # pattern pour charger les 385 fichiers

# Paths pour sauvegarde des modèles / artefacts
PATH_SAVE_CF_MODEL       = "model_cf.pkl"
PATH_SAVE_CB_MODEL       = "model_cb.pkl"
PATH_SAVE_DATA_TRANSFORM = "data_transform.pkl"  # pour tout ce qui est nécessaire (matrices, etc.)

# Nombre de recommandations à retourner
TOP_N = 5

# Fonctions Utilitaires

In [3]:
def load_click_data(clicks_path_pattern: str) -> pd.DataFrame:
    """
    Charge les fichiers de clics (user interactions) stockés dans plusieurs CSV 
    et les concatène en un unique DataFrame.

    Paramètres:
    -----------
    - clicks_path_pattern : str
        Chemin ou pattern glob pour trouver les fichiers CSV des clics.

    Retour:
    -------
    - pd.DataFrame
        DataFrame contenant toutes les interactions (tous les clics).
    """
    all_files = glob.glob(clicks_path_pattern)
    df_list = []

    for filename in all_files:
        df_temp = pd.read_csv(filename)
        df_list.append(df_temp)

    df_clicks = pd.concat(df_list, ignore_index=True)
    return df_clicks

In [4]:
def load_articles_data(embeddings_path: str, metadata_path: str) -> tuple[np.ndarray, pd.DataFrame]:
    """
    Charge la matrice d'embeddings des articles ainsi que leurs métadonnées.

    Paramètres:
    -----------
    - embeddings_path : str
        Chemin vers le fichier pickle contenant un numpy.ndarray des embeddings.
    - metadata_path : str
        Chemin vers le fichier CSV contenant les métadonnées (article_id, category_id, etc.).

    Retour:
    -------
    - (np.ndarray, pd.DataFrame)
        - Matrice d'embeddings (shape (364047, 250))
        - DataFrame des métadonnées (364047 lignes, 5 colonnes)
    """
    # Chargement des embeddings
    with open(embeddings_path, 'rb') as f:
        articles_embeddings = pickle.load(f)  # type: np.ndarray
    # Chargement du CSV de métadonnées
    articles_metadata = pd.read_csv(metadata_path)
    return articles_embeddings, articles_metadata

In [5]:
def preprocess_click_data(df_clicks: pd.DataFrame) -> pd.DataFrame:
    """
    Exécute un prétraitement minimal sur les données de clics :
    - Conversion de certaines colonnes (si nécessaire)
    - Tri / suppression de colonnes inutiles (facultatif)
    - Filtrage éventuel d'anomalies

    Paramètres:
    -----------
    - df_clicks : pd.DataFrame
        DataFrame des clics déjà concaténés.

    Retour:
    -------
    - pd.DataFrame
        DataFrame pré-traité.
    """

    # Dans cet exemple, on va simplement s'assurer qu'on a bien user_id et click_article_id
    df_clicks = df_clicks.dropna(subset=['user_id', 'click_article_id'])  # enlever lignes incomplètes
    df_clicks['user_id'] = df_clicks['user_id'].astype(int)
    df_clicks['click_article_id'] = df_clicks['click_article_id'].astype(int)

    return df_clicks

# Chargement des données

In [6]:
print("Chargement des données...")

# Chargement des embeddings et métadonnées
articles_embeddings, articles_metadata = load_articles_data(PATH_ARTICLES_EMBEDDINGS, PATH_ARTICLES_METADATA)

# Chargement des logs de clic
df_clicks_raw = load_click_data(PATH_CLICKS_PATTERN)

# Prétraitement
df_clicks = preprocess_click_data(df_clicks_raw)

print("Taille du DataFrame de clics :", df_clicks.shape)
print("Taille de la matrice d'embeddings :", articles_embeddings.shape)
print("Taille du DataFrame de métadonnées :", articles_metadata.shape)

Chargement des données...
Taille du DataFrame de clics : (2988181, 12)
Taille de la matrice d'embeddings : (364047, 250)
Taille du DataFrame de métadonnées : (364047, 5)


# Préparation du jeu de données

In [7]:
# Combien d'articles distincts dans df_clicks ?
unique_articles_in_clicks = df_clicks['click_article_id'].unique()
print("Nombre d'articles distincts dans df_clicks :", len(unique_articles_in_clicks))

# Vérifier si certains IDs sont hors [0..364047)
out_of_range_ids = unique_articles_in_clicks[
    (unique_articles_in_clicks < 0) | (unique_articles_in_clicks >= articles_embeddings.shape[0])
]
if len(out_of_range_ids) > 0:
    print(f"ATTENTION: {len(out_of_range_ids)} article_ids sont hors de la plage [0..364047).")
    print("Exemple d'IDs hors range :", out_of_range_ids[:20])
else:
    print("Aucun article_id hors de la plage [0..364047).")

Nombre d'articles distincts dans df_clicks : 46033
Aucun article_id hors de la plage [0..364047).


Nous allons créer un DataFrame user-article, avec le nombre de clics par (user, article).
Dans le cas d'un CF basé sur les interactions implicites, la "note" peut être le nombre de clics (ou 1 s'il y a au moins un clic).
Ici, nous allons simplifier et compter le nombre total de clics user->article.

In [8]:
print("Construction de la matrice (sparse) user-article...")

# GroupBy pour obtenir (user_id, article_id, click_count)
df_user_item = (
    df_clicks
    .groupby(['user_id', 'click_article_id'])['click_timestamp']
    .count()
    .reset_index()
    .rename(columns={'click_timestamp': 'click_count'})
)

print("Taille df_user_item :", df_user_item.shape)
print("Nombre d'utilisateurs (df_user_item) :", df_user_item['user_id'].nunique())
print("Nombre d'articles (df_user_item) :", df_user_item['click_article_id'].nunique())

# Liste unique des user_id et article_id
user_ids = df_user_item["user_id"].unique()
article_ids = df_user_item["click_article_id"].unique()

# On crée des maps pour passer de l'ID réel à un index consécutif
user_id_to_index = {uid: i for i, uid in enumerate(user_ids)}
article_id_to_index = {aid: i for i, aid in enumerate(article_ids)}

# Construction des listes 'row', 'col', 'data' pour la COO matrix
row = [user_id_to_index[uid] for uid in df_user_item["user_id"]]
col = [article_id_to_index[aid] for aid in df_user_item["click_article_id"]]
data = df_user_item["click_count"].values

# Création de la matrice en format COO puis conversion en CSR
sparse_user_item_matrix = coo_matrix(
    (data, (row, col)),
    shape=(len(user_ids), len(article_ids))
).tocsr()

# Vérification
print("Taille de la matrice sparse (CSR) :", sparse_user_item_matrix.shape)

# Vérifier s'il manque des articles par rapport à df_clicks (normalement non).
missing_from_matrix = set(unique_articles_in_clicks) - set(article_ids)
print(f"Nb d'articles présents dans df_clicks mais pas dans la matrice pivot: {len(missing_from_matrix)}")
if len(missing_from_matrix) > 0:
    print("Exemple d'articles manquants:", list(missing_from_matrix)[:20])
    print("=> Possibles raisons : filtrage, ou IDs hors range, etc.")

Construction de la matrice (sparse) user-article...
Taille df_user_item : (2950710, 3)
Nombre d'utilisateurs (df_user_item) : 322897
Nombre d'articles (df_user_item) : 46033
Taille de la matrice sparse (CSR) : (322897, 46033)
Nb d'articles présents dans df_clicks mais pas dans la matrice pivot: 0


# Implémentation du CF k-NN

In [9]:
def fit_cf_model_knn(sparse_matrix, user_ids, article_ids):
    """
    Entraîne un modèle user-based CF avec NearestNeighbors sur la matrice sparse (CSR).
    Retourne un dictionnaire contenant le modèle et les infos nécessaires.

    Paramètres:
    -----------
    - sparse_matrix : csr_matrix
        Matrice (n_users x n_articles) contenant les interactions (count).
    - user_ids : np.ndarray
        Tableau des user_id uniques (indexés dans la matrice).
    - article_ids : np.ndarray
        Tableau des article_id uniques (indexés dans la matrice).

    Retour:
    -------
    - dict
        Contient 'knn_model', 'user_ids', 'article_ids' etc.
    """
    knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
    knn_model.fit(sparse_matrix)

    model_cf = {
        'knn_model': knn_model,
        'user_ids': user_ids,
        'article_ids': article_ids,
        'user_id_to_index': user_id_to_index,
        'article_id_to_index': article_id_to_index,
        'sparse_matrix': sparse_matrix
    }
    return model_cf

In [10]:
def predict_cf_knn(user_id, model_cf, sparse_matrix, top_n=5, k_neighbors=5):
    """
    Recommande des articles à un utilisateur donné en utilisant le modèle k-NN user-based.

    Paramètres:
    -----------
    - user_id : int
        Identifiant "réel" de l'utilisateur (pas l'indice interne)
    - model_cf : dict
        Dictionnaire contenant le modèle kNN et les tables de correspondance
    - sparse_matrix : csr_matrix
        Matrice user-item (n_users x n_articles)
    - top_n : int
        Nombre d'articles recommandés
    - k_neighbors : int
        Nombre de voisins les plus proches à agréger

    Retour:
    -------
    - list
        Liste d'article_id recommandés (jusqu'à top_n).
    """
    knn_model = model_cf['knn_model']
    all_user_ids = model_cf['user_ids']
    all_article_ids = model_cf['article_ids']
    u2i = model_cf['user_id_to_index']

    if user_id not in u2i:
        # cold start => aucune reco
        return []

    # Récupérer l'index interne du user
    user_idx = u2i[user_id]

    # On récupère la ligne correspondant à cet utilisateur => shape (1, nb_articles) 
    user_vector = sparse_matrix[user_idx]

    # Recherche des k voisins (on demande k+1 au cas où le user lui-même est dans les résultats)
    distances, indices = knn_model.kneighbors(user_vector, n_neighbors=k_neighbors+1)

    # indices[0] = [user_idx, neighbor1, neighbor2, ...]
    neighbor_indices = indices[0].tolist()

    # Exclure l'utilisateur lui-même si présent
    if user_idx in neighbor_indices:
        neighbor_indices.remove(user_idx)

    # S'assurer d'avoir k voisins max
    neighbor_indices = neighbor_indices[:k_neighbors]

    # On récupère la sous-matrice pour ces voisins => shape (k_neighbors, nb_articles)
    neighbors_matrix = sparse_matrix[neighbor_indices]

    # On somme leurs usages d'article
    article_scores = neighbors_matrix.sum(axis=0).A1  # .A1 => convertit en array 1D

    # Exclure les articles déjà vus par l'utilisateur
    already_viewed = user_vector.indices  # colonnes non nulles pour cet utilisateur
    article_scores[already_viewed] = -9999  # on met un score très bas pour ignorer

    # On récupère les top_n articles
    top_article_indices = np.argsort(-article_scores)[:top_n]

    # On convertit ces indices internes en article_id
    recommended_articles = [all_article_ids[i] for i in top_article_indices]

    return recommended_articles

In [11]:
print("\nEntraînement du modèle CF (k-NN)...")
model_cf = fit_cf_model_knn(sparse_user_item_matrix, user_ids, article_ids)


Entraînement du modèle CF (k-NN)...


# Implémentation Content-Based

In [12]:
def build_user_profiles(articles_embeddings: np.ndarray,
                        df_clicks: pd.DataFrame) -> dict:
    """
    Construit un profil d'utilisateur sous la forme d'un embedding moyen 
    des articles qu'il a consultés.

    Paramètres:
    -----------
    - articles_embeddings : np.ndarray
        Matrice d'embeddings des articles (shape = (nb_articles, dims))
        L'index i correspond à article_id = i (important !).
    - df_clicks : pd.DataFrame
        DataFrame des clics, contenant user_id et click_article_id.

    Retour:
    -------
    - dict
        Clé : user_id
        Valeur : np.ndarray (embedding moyen du user)
    """
    user_profiles = {}
    user_articles_map = df_clicks.groupby('user_id')['click_article_id'].apply(list)

    for uid, articles_list in user_articles_map.items():
        # Filtre si out-of-range
        valid_articles = [a for a in articles_list if 0 <= a < len(articles_embeddings)]
        if len(valid_articles) == 0:
            continue
        emb = articles_embeddings[valid_articles].mean(axis=0)
        user_profiles[uid] = emb
    return user_profiles

In [13]:
def recommend_content_based(user_id: int,
                            user_profiles: dict,
                            articles_embeddings: np.ndarray,
                            top_n: int = TOP_N) -> list:
    """
    Recommande des articles pour un user_id donné en comparant son embedding moyen 
    à tous les articles via la similarité cosinus.

    Paramètres:
    -----------
    - user_id : int
        Identifiant utilisateur
    - user_profiles : dict
        Clé : user_id, Valeur : embedding moyen (np.ndarray)
    - articles_embeddings : np.ndarray
        Matrice d'embeddings de tous les articles (shape = (nb_articles, dims))
    - top_n : int
        Nombre de recommandations à renvoyer

    Retour:
    -------
    - list
        Liste des article_id recommandés (jusqu'à top_n).
    """
    if user_id not in user_profiles:
        # cold start => on ne peut pas faire de reco content-based
        return []

    user_emb = user_profiles[user_id].reshape(1, -1)  # shape(1, dims)

    # Calcul de similarité cosinus avec tous les articles
    sims = cosine_similarity(user_emb, articles_embeddings)[0]  # shape(nb_articles,)

    # Tri descendant
    sorted_indices = np.argsort(-sims)

    # Top_n
    return sorted_indices[:top_n].tolist()

In [14]:
# Construction des profils utilisateurs sur le df complet (pour la démo)
print("\nConstruction des profils utilisateurs pour le Content-Based...")
user_profiles_cb = build_user_profiles(articles_embeddings, df_clicks)
model_cb = {
    'user_profiles': user_profiles_cb,
    'articles_embeddings': articles_embeddings
}


Construction des profils utilisateurs pour le Content-Based...


# Sauvegarde des modèles

In [15]:
# Sauvegarde de CF et CB dans des pickles
print("Sauvegarde des modèles et des données transformées...")

with open(PATH_SAVE_CF_MODEL, 'wb') as f:
    pickle.dump(model_cf, f)

with open(PATH_SAVE_CB_MODEL, 'wb') as f:
    pickle.dump(model_cb, f)

# Sauvegarde des dimensions de la matrice (optionnel)
data_transform = {
    "nb_users": sparse_user_item_matrix.shape[0],
    "nb_articles": sparse_user_item_matrix.shape[1]
}

with open("data_transform.pkl", "wb") as f:
    pickle.dump(data_transform, f)

print("Modèles et data_transform sauvegardés avec succès.")

Sauvegarde des modèles et des données transformées...
Modèles et data_transform sauvegardés avec succès.


# Exemple d'utilisation

Dans un environnement Streamlit ou Azure Functions, on peut charger ces pickles 
et appeler les fonctions de prédiction ci-dessous.

In [16]:
def load_models(cf_model_path: str, cb_model_path: str):
    """
    Charge les modèles CF et CB depuis des fichiers pickle.
    """
    with open(cf_model_path, 'rb') as f:
        model_cf_loaded = pickle.load(f)
    with open(cb_model_path, 'rb') as f:
        model_cb_loaded = pickle.load(f)
    return model_cf_loaded, model_cb_loaded

In [17]:
def get_recommendations(user_id: int, 
                        model_cf: dict, 
                        sparse_matrix: csr_matrix,
                        model_cb: dict, 
                        method: str = 'cf', 
                        top_n: int = TOP_N) -> list:
    """
    Retourne une liste de recommandations pour un utilisateur donné, 
    selon la méthode choisie : 'cf' ou 'content'.

    Paramètres:
    -----------
    - user_id : int
        Identifiant utilisateur
    - model_cf : dict
        Modèle CF (k-NN) chargé
    - sparse_matrix : csr_matrix
        Matrice user-item
    - model_cb : dict
        Modèle Content-Based chargé
    - method : str
        'cf' ou 'content'
    - top_n : int
        Nombre de recommandations

    Retour:
    -------
    - list
        Liste d'articles recommandés
    """
    if method == 'cf':
        return predict_cf_knn(user_id, model_cf, sparse_matrix, top_n=top_n)
    elif method == 'content':
        user_profiles_cb = model_cb['user_profiles']
        articles_embeddings_cb = model_cb['articles_embeddings']
        return recommend_content_based(user_id, user_profiles_cb, articles_embeddings_cb, top_n=top_n)
    else:
        raise ValueError("Méthode inconnue. Choisir 'cf' ou 'content'.")

Exemple d'appel
(Dans la pratique, on chargerait d'abord les modèles, puis on exécuterait get_recommendations() pour chaque requête.)

In [18]:
example_user_id = 12345
cf_recos = get_recommendations(example_user_id, model_cf, csr_matrix(sparse_user_item_matrix), model_cb, method='cf', top_n=TOP_N)
cb_recos = get_recommendations(example_user_id, model_cf, csr_matrix(sparse_user_item_matrix), model_cb, method='content', top_n=TOP_N)

print(f"Recommandations CF pour l'utilisateur {example_user_id} : {cf_recos}")
print(f"Recommandations CB pour l'utilisateur {example_user_id} : {cb_recos}")

Recommandations CF pour l'utilisateur 12345 : [np.int64(96210), np.int64(284985), np.int64(194920), np.int64(5278), np.int64(236566)]
Recommandations CB pour l'utilisateur 12345 : [111862, 95246, 231146, 133356, 84624]
